In [1]:
import torch
import torch.nn as nn

from PIL import Image
import time

from mss.linux import MSS as mss
import cv2 as cv
import numpy as np

# Load YOLOv5 with PyTorch Hub from Ultralytics
model = torch.hub.load("ultralytics/yolov5", "custom", path="csgo-detection-v3.pt")

if torch.cuda.is_available():
    model.cuda()

Using cache found in /home/dalai/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-6-4 Python-3.10.1 torch-1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 367 layers, 46135203 parameters, 0 gradients
Adding AutoShape... 


In [17]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
        self.coordinates = np.array([x, y])

    def __sub__(self, other) -> float:
        """Calculates euclidean distance between two Points."""
        return np.linalg.norm(self.coordinates - other.coordinates)

    def __repr__(self) -> str:
        return f"Point(x={self.x}, y={self.y})"


class Target(Point):
    def __init__(self, target: torch.Tensor) -> None:
        self.x: float = target[0].item()
        self.y: float = target[1].item()
        self.w: float = target[2].item()
        self.h: float = target[3].item()
        self.confidence: float = target[4].item()
        self.label: int = int(target[5].item())
        self.tensor: torch.Tensor = target
        self.coordinates = np.array([self.x, self.y])

    # def __lt__(self, other)

    def __repr__(self) -> str:
        return f"Target({self.label}: x={self.x}, y={self.y})"

In [34]:
img = Image.open("csgo.mp4-out000014.png")


In [35]:
results = model(img)

In [36]:
def get_targets(results: list, labels: list[int]) -> list[Target]:
    """Returns a list with results filtered by the classes specified in the labels argument."""
    return [Target(target) for target in results.xywh[0] if target[5] in labels]

def sort_targets(targets: list[Target], aim_center: Point) -> list[Target]:
    return sorted(targets, key=lambda target: aim_center - target)

In [37]:
AIM_CENTER = Point(x=960, y=540)
AIM_CENTER

Point(x=960, y=540)

In [28]:
targets = get_targets(results, labels=[2,3])
targets

[Target(3: x=647.32275390625, y=174.81887817382812),
 Target(3: x=201.32089233398438, y=181.86473083496094),
 Target(3: x=1105.286376953125, y=166.70626831054688),
 Target(3: x=687.9887084960938, y=160.46310424804688)]

In [ ]:
for target in targets:
    print(AIM_CENTER - target)

In [29]:
t1 = targets[1]
t1

Target(3: x=201.32089233398438, y=181.86473083496094)

Point(x=960, y=540)

In [18]:
t1 - AIM_CENTER

TypeError: unsupported operand type(s) for -: 'Target' and 'Point'

In [31]:
sorted_targets = sort_targets(targets, AIM_CENTER)
sorted_targets

[Target(3: x=1105.286376953125, y=166.70626831054688),
 Target(3: x=687.9887084960938, y=160.46310424804688),
 Target(3: x=647.32275390625, y=174.81887817382812),
 Target(3: x=201.32089233398438, y=181.86473083496094)]

In [32]:
for target in sorted_targets:
    print(AIM_CENTER - target)

400.57002065407113
466.94581906539435
480.75389958168086
838.9605827622754


In [8]:
for target in targets:
    print(target)